In [28]:
import onnxruntime as ort
print(ort.get_available_providers())

['AzureExecutionProvider', 'CPUExecutionProvider']


In [ ]:
from inference import get_model
import supervision as sv
import cv2
from dotenv import load_dotenv
import os

# Load environment variables and API key
load_dotenv()
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

# Load your trained model from Roboflow
model = get_model(model_id="test-ns29a/3", api_key=ROBOFLOW_API_KEY)

# Initialize webcam (0 = default webcam)
cap = cv2.VideoCapture(0)

# Supervision annotators
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Run inference
    results = model.infer(frame)[0]
    
    # Convert results to Supervision Detections
    detections = sv.Detections.from_inference(results)
    
    # Annotate frame
    annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)
    
    # Show frame
    cv2.imshow("Real-time Detection", annotated_frame)
    
    # Break when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


[12/12/25 22:01:06] ERROR    Unknown resize method, defaulting to 'Stretch to' - this may result in ]8;id=333023;file://c:\Users\Shane\AppData\Local\Programs\Python\Python312\Lib\site-packages\inference\core\models\roboflow.py\roboflow.py]8;;\:]8;id=15516;file://c:\Users\Shane\AppData\Local\Programs\Python\Python312\Lib\site-packages\inference\core\models\roboflow.py#478\478]8;;\
                             degraded model performance.                                                           

KeyboardInterrupt: 

: 